In [3]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from skimage import io
from sklearn import svm
from sklearn.metrics import accuracy_score
import cv2
from skimage.feature import graycomatrix, graycoprops
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from tqdm import tqdm
from PIL import Image, ImageEnhance
import time

In [5]:
#uncomment the following line if hinge_and_cold directory is not included in the project
# !git clone https://github.com/Hero2323/hinge_and_cold_feature_extraction.git
os.chdir('hinge_and_cold_feature_extraction')
os.chdir('src')
#!pip install imutils
import hinge_feature_extraction as hinge
import cold_feature_extraction as cold
os.chdir('../../')

In [8]:
# 
male_paths = os.listdir('./input/cmp23-handwritten-males-vs-females/Males/Males')
female_paths = os.listdir('./input/cmp23-handwritten-males-vs-females/Females/Females')
image_labels = []
image_paths = []
for i in range(len(male_paths)):
    if male_paths[i] == 'M152.jpg':
        continue
    image_paths.append(os.path.join('./input/cmp23-handwritten-males-vs-females/Males/Males', male_paths[i]))
    image_labels.append(0)
for i in range(len(female_paths)):
    if female_paths[i] == 'F87.jpg':
        continue
    image_paths.append(os.path.join('./input/cmp23-handwritten-males-vs-females/Females/Females', female_paths[i]))
    image_labels.append(1)

In [9]:
# Split the paths into train and test
from sklearn.model_selection import train_test_split
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, image_labels, test_size=0.25, random_state=42)
print("Toal Train size: {}".format(len(train_paths)))
print("Test size: {}".format(len(test_paths)))

Toal Train size: 270
Test size: 91


In [10]:
# Read the image and apply thresholding to it
# Then apply morphological operations to get rid of the noise
train_images, test_images = [], []
for i in tqdm(range(len(train_paths))): #use skimage's imread
    try:
        img = cv2.imread(train_paths[i], 0)
#         kernel = np.ones((3, 3))
#         img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
#         img = cv2.medianBlur(img, 3)
#         (thresh, img) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        train_images.append(img)
    except  Exception as e:
        print(e , 'in - ', train_paths[i])
        break
for i in tqdm(range(len(test_paths))): #use skimage's imread
    try:
        img = cv2.imread(test_paths[i], 0)
#         kernel = np.ones((3, 3))
#         img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
#         img = cv2.medianBlur(img, 3)
#         (thresh, img) = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        test_images.append(img)
    except  Exception as e:
        print(e , 'in - ', test_paths[i])
        break


100%|██████████| 91/91 [00:17<00:00,  5.10it/s]


In [11]:
# GLCM Classifier
def get_glcm_feature(img_list):
    features = []
    glcm_texture_features = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
    used_features = ['energy','homogeneity']
    for img in tqdm(img_list):
        glcm = graycomatrix(img, distances=[1], angles=[0, np.pi / 2, np.pi, 3 * np.pi / 2], levels=256, normed=True)
        curr_img_features = []
        for feature in used_features:
            extracted_feature = np.array(graycoprops(glcm, feature), dtype='float64').flatten()
            curr_img_features.append(np.sum(extracted_feature))
        features.append(np.array(curr_img_features, dtype='float64').flatten())
    features = np.array(features, dtype='float64')
    return features
train_glcm = get_glcm_feature(train_images)
test_glcm = get_glcm_feature(test_images)

100%|██████████| 91/91 [00:17<00:00,  5.15it/s]


In [12]:
# GLCM Using SVM
glcm_svm = svm.SVC()
glcm_svm.fit(train_glcm, train_labels)
glcm_svm_predictions = glcm_svm.predict(test_glcm)
accuracy_score(test_labels, glcm_svm_predictions)

0.6263736263736264

In [22]:
# GLCM Using Random Forest Classifier
glcm_rfc = RandomForestClassifier(n_estimators= 100)
glcm_rfc.fit(train_glcm, train_labels)
glcm_rfc_predictions = glcm_rfc.predict(test_glcm)
accuracy_score(test_labels, glcm_rfc_predictions)

0.5384615384615384

In [23]:
# Hinge Classifier
hinge_classifier = hinge.Hinge((10, 3, False, True))
train_hinge_features = []
for img in tqdm(train_images):
    train_hinge_feature = hinge_classifier.get_hinge_features(img)
    train_hinge_features.append(train_hinge_feature)
test_hinge_features = []
for img in tqdm(test_images):
    test_hinge_feature = hinge_classifier.get_hinge_features(img)
    test_hinge_features.append(test_hinge_feature)

100%|██████████| 91/91 [01:09<00:00,  1.30it/s]


In [24]:
# Hinge using SVM
hinge_svm = svm.SVC()
hinge_svm.fit(train_hinge_features, train_labels)
hinge_rfc_predictions = hinge_svm.predict(test_hinge_features)
accuracy_score(test_labels, hinge_rfc_predictions)

0.6263736263736264

In [27]:
# Hinge using Random Forest Classifier
hinge_rfc = RandomForestClassifier(n_estimators= 1000)
hinge_rfc.fit(train_hinge_features, train_labels)
hinge_rfc_predictions = hinge_rfc.predict(test_hinge_features)
accuracy_score(test_labels, hinge_rfc_predictions)

0.7582417582417582

In [28]:
# Cold Classifier
cold_classifier = cold.Cold((10, 3, False, True))
train_cold_features = []
for img in tqdm(train_images):
    train_cold_feature = cold_classifier.get_cold_features(img)
    train_cold_features.append(train_cold_feature)
test_cold_features = []
for img in tqdm(test_images):
    test_cold_feature = cold_classifier. get_cold_features(img)
    test_cold_features.append(test_cold_feature)

  0%|          | 0/270 [00:00<?, ?it/s]d:\GitHub projects\Handwriting-Based-Gender-Classifier\hinge_and_cold_feature_extraction\src\cold_feature_extraction.py:123: RuntimeWarning: divide by zero encountered in log10
  rhos_log_space = np.log10(rhos)
100%|██████████| 91/91 [01:19<00:00,  1.14it/s]


In [32]:
# Cold using SVM
cold_svm = svm.SVC()
cold_svm.fit(train_hinge_features, train_labels)
hinge_rfc_predictions = hinge_svm.predict(test_hinge_features)
accuracy_score(test_labels, hinge_rfc_predictions)

0.6263736263736264

In [33]:
# Cold using Random Forest Classifer
hinge_rfc = RandomForestClassifier(n_estimators= 1000)
hinge_rfc.fit(train_hinge_features, train_labels)
hinge_rfc_predictions = hinge_rfc.predict(test_hinge_features)
accuracy_score(test_labels, hinge_rfc_predictions)

0.7582417582417582

In [ ]:
# import shutil
# os.mkdir('test')
# for path in test_paths:
#     shutil.copy(path, 'test')

In [34]:
if os.path.exists('test'):
    test_img_paths = []
    prediction_file = open('results.txt', 'a', buffering=1)
    timing_file = open('time.txt', 'a', buffering=1)
    for img in os.listdir('test'):
        test_img_paths.append('test/' + img)
    for img_path in test_img_paths:
        test_img = cv2.imread(img_path, 0)
        start = time.time()
        prediction = 1 - hinge_rfc.predict(np.reshape(hinge_classifier.get_hinge_features(test_img), (1, 780)))
        duration = time.time() - start
        if duration == 0 :
            duration = 0.001
        prediction_file.write(str(prediction[0]) + '\n')
        timing_file.write(str(round(duration, 2)) + '\n')
    prediction_file.close()
    timing_file.close()

In [ ]:
# # Crop the white parts of the image by storing the index of the first (top left) black pixel and
# # the index of the last (bottom right) black pixel
# first_black_idx = [1e6, 1e6]
# last_black_idx = [0, 0]
# # first, get the idx of the top left black pixel
# # for i in range(len(train_images)):
# for i in range(1):
#     i = 1
#     done_first = False
#     done_second = False
#     for j in range(train_images[i].shape[0]):
#         for k in range(train_images[i].shape[1]):
#             if train_images[i][j][k] != 255:
#                 first_black_idx[0] = j
#                 first_black_idx[1] = k
#                 done_first = True
#                 break
#         if done_first:
#             break
#     for j in range(train_images[i].shape[0] - 1, 0, -1):
#         for k in range(train_images[i].shape[1] - 1, 0, -1):
#             if train_images[i][j][k] != 255:
#                 last_black_idx[0] = j
#                 last_black_idx[1] = k
#                 done_second = True
#                 break
#         if done_second:
#             break
